**_Disclaimer:_** This is a copy of [Peter Norvig](https://en.wikipedia.org/wiki/Peter_Norvig)'s essay ["How to Write a Spelling Corrector"](http://norvig.com/spell-correct.html) in form of a Jupyter notebook for educational purposes. Following the link to the original, you can find further reading, acknowledgments, and references to other natural and computer languages.

----
    
# How to Write a Spelling Corrector
    
One week in 2007, two friends (Dean and Bill) independently told me they were amazed at Google's spelling correction. Type in a search like [speling](http://www.google.com/search?q=speling) and Google instantly comes back with **Showing results for:** [**_spelling_**](http://www.google.com/search?q=spelling). I thought Dean and Bill, being highly accomplished engineers and mathematicians, would have good intuitions about how this process works. But they didn't, and come to think of it, why should they know about something so far outisde their specialty?

I figured they, and others, could benefit from an explanation. The full details of an industrial-strength spell corrector are quite complex (you can read a little about it [here](http://static.googleusercontent.com/external_content/untrusted_dlcp/research.google.com/en/us/pubs/archive/36180.pdf) or [here](http://citeseerx.ist.psu.edu/viewdoc/download;jsessionid=52A3B869596656C9DA285DCE83A0339F?doi=10.1.1.146.4390&rep=rep1&type=pdf)). But I figured that in the course of a transcontinental plane ride I could write and explain a toy spelling corrector that achieves 80 or 90% accuracy at a processing speed of at least 10 words per second in about half a page of code.

And here it is (or see [spell.py](http://norvig.com/spell.py)): 

In [4]:
import re ## regular expressions
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

The function ```correction(word)``` returns a likely spelling correction: 

In [2]:
correction('speling')

'spelling'

In [3]:
correction('korrectud')

'corrected'

### How It Works: Some Probability Theory

The call ```correction(w)``` tries to choose the most likely spelling correction for ```w```. There is no way to know for sure (for example, should "lates" be corrected to "late" or "latest" or "lattes" or ...?), which suggests we use probabilities. We are trying to find the correction $c$, out of all possible candidate corrections, that maximizes the probability that $c$ is the intended correction, given the original word $w$:

$$
    \mathrm{argmax}_{c \,\in\, candidates} P(c\,|\,w) 
$$    

By Bayes' Theorem this is equivalent to:

$$
    \mathrm{argmax}_{c \,\in\, candidates} P(c) \, P(w\,|\,c) \,/\, P(w) 
$$

Since $P(w)$ is the same for every possible candidate $c$, we can factor it out, giving:

$$
    \mathrm{argmax}_{c \,\in\, candidates} P(c) \, P(w\,|\,c) 
$$

The four parts of this expression are:

1. **Selection Mechanism:** $\mathrm{argmax}$
    <br>
    We choose the candidate with the highest combined probability.
    <br><br>
2. **Candidate Model:** $c \,\in\, candidates$
    <br>
    This tells us which candidate corrections, $c$, to consider.
    <br><br>
3. **Language Model:** $P(c)$
    <br>
    The probability that $c$ appears as a word of English text. For example, occurrences of "the" make up about 7% of English text, so we should have $P(the) = 0.07$.
    <br><br>
4. **Error Model:** $P(w\,|\,c)$
    <br>
    The probability that $w$ would be typed in a text when the author meant $c$. For example, $P(teh\,|\,the)$ is relatively high, but $P(theeexyz\,|\,the)$ would be very low.

One obvious question is: why take a simple expression like $P(c\,|\,w)$ and replace it with a more complex expression involving two models rather than one? The answer is that $P(c\,|\,w)$ is already conflating two factors, and it is easier to separate the two out and deal with them explicitly. Consider the misspelled word $w$="thew" and the two candidate corrections $c$="the" and $c$="thaw". Which has a higher $P(c\,|\,w)$? Well, "thaw" seems good because the only change is "a" to "e", which is a small change. On the other hand, "the" seems good because "the" is a very common word, and while adding a "w" seems like a larger, less probable change, perhaps the typist's finger slipped off the "e". The point is that to estimate $P(c\,|\,w)$ we have to consider both the probability of $c$ and the probability of the change from $c$ to $w$ anyway, so it is cleaner to formally separate the two factors.

### How It Works: Some Python

The four parts of the program are:

1\. **Selection Mechanism:** In Python, ```max``` with a ```key``` argument does '$\mathrm{argmax}$'.

2\. **Candidate Model:** First a new concept: a **simple edit** to a word is a deletion (remove one letter), a transposition (swap two adjacent letters), a replacement (change one letter to another) or an insertion (add a letter). The function ```edits1``` returns a set of all the edited strings (whether words or not) that can be made with one simple edit: 

In [1]:
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

This can be a big set. For a word of length $n$, there will be $n$ deletions, $n-1$ transpositions, $26n$ alterations, and $26(n+1)$ insertions, for a total of $54n+25$ (of which a few are typically duplicates). For example, 

In [2]:
len(edits1('somthing'))

442

However, if we restrict ourselves to words that are _known_ - that is, in the dictionary - then the set is much smaller: 

In [5]:
def known(words): return set(w for w in words if w in WORDS)

known(edits1('somthing'))

{'something', 'soothing'}

We'll also consider corrections that require two simple edits. This generates a much bigger set of possibilities, but usually only a few of them are known words: 

In [13]:
def edits2(word): return (e2 for e1 in edits1(word) for e2 in edits1(e1))

len(set(edits2('something')))

114324

In [14]:
known(edits2('somthing'))

{'loathing',
 'nothing',
 'scathing',
 'seething',
 'smoothing',
 'something',
 'soothing',
 'sorting'}

We say that the results of ```edits2(w)``` have an **edit distance** of 2 from ```w```.

3\. **Language Model:** We can estimate the probability of a word, ```P(word)```, by counting the number of times each word appears in a text file of about a million words, [big.txt](http://norvig.com/big.txt). It is a concatenation of public domain book excerpts from [Project Gutenberg](http://www.gutenberg.org/wiki/Main_Page) and lists of most frequent words from [Wiktionary](http://en.wiktionary.org/wiki/Wiktionary:Frequency_lists) and the [British National Corpus](http://www.kilgarriff.co.uk/bnc-readme.html). The function ```words``` breaks text into words, then the variable ```WORDS``` holds a Counter of how often each word appears, and ```P``` estimates the probability of each word, based on this Counter: 

In [15]:
def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): return WORDS[word] / N

We can see that there are 32,198 distinct words, which together appear 1,115,585 times, with 'the' being the most common word, appearing 79,809 times (or a probability of about 7%) and other words being less probable: 

In [22]:
len(WORDS)

32198

In [23]:
sum(WORDS.values())

1115585

In [24]:
WORDS.most_common(10)

[('the', 79809),
 ('of', 40024),
 ('and', 38312),
 ('to', 28765),
 ('in', 22023),
 ('a', 21124),
 ('that', 12512),
 ('he', 12401),
 ('was', 11410),
 ('it', 10681)]

In [18]:
max(WORDS, key=P)

'the'

In [19]:
P('the')

0.07154004401278254

In [20]:
P('unmentioned')

0.0

4\. **Error Model:** When I started to write this program, sitting on a plane in 2007, I had no data on spelling errors, and no internet connection (I know that may be hard to imagine today). Without data I couldn't build a good spelling error model, so I took a shortcut: I defined a trivial, flawed error model that says all known words of edit distance 1 are infinitely more probable than known words of edit distance 2, and infinitely less probable than a known word of edit distance 0. So we can make ```candidates(word)``` produce the first non-empty list of candidates in order of priority:

1. The original word, if it is known; otherwise
2. The list of known words at edit distance one away, if there are any; otherwise
3. The list of known words at edit distance two away, if there are any; otherwise
4. The original word, even though it is not known. 

Then we don't need to multiply by a $P(w\,|\,c)$ factor, because every candidate at the chosen priority will have the same probability (according to our flawed model). That gives us: 

In [25]:
def correction(word): return max(candidates(word), key=P)

def candidates(word): return known([word]) or known(edits1(word)) or known(edits2(word)) or [word]

### Evaluation

Now it is time to evaluate how well this program does. After my plane landed, I downloaded Roger Mitton's (Birkbeck spelling error corpus)[http://ota.ahds.ac.uk/texts/0643.html] from the Oxford Text Archive. From that I extracted two test sets of corrections. The first is for development, meaning I get to look at it while I'm developing the program. The second is a final test set, meaning I'm not allowed to look at it, nor change my program after evaluating on it. This practice of having two sets is good hygiene; it keeps me from fooling myself into thinking I'm doing better than I am by tuning the program to one specific set of tests. I also wrote some unit tests: 

In [27]:
def unit_tests():
    assert correction('speling') == 'spelling'              # insert
    assert correction('korrectud') == 'corrected'           # replace 2
    assert correction('bycycle') == 'bicycle'               # replace
    assert correction('inconvient') == 'inconvenient'       # insert 2
    assert correction('arrainged') == 'arranged'            # delete
    assert correction('peotry') =='poetry'                  # transpose
    assert correction('peotryy') =='poetry'                 # transpose + delete
    assert correction('word') == 'word'                     # known
    assert correction('quintessential') == 'quintessential' # unknown
    assert words('This is a TEST.') == ['this', 'is', 'a', 'test']
    assert Counter(words('This is a test. 123; A TEST this is.')) == (
           Counter({'123': 1, 'a': 2, 'is': 2, 'test': 2, 'this': 2}))
    assert len(WORDS) == 32198
    assert sum(WORDS.values()) == 1115585
    assert WORDS.most_common(10) == [
        ('the', 79809),
        ('of', 40024),
        ('and', 38312),
        ('to', 28765),
        ('in', 22023),
        ('a', 21124),
        ('that', 12512),
        ('he', 12401),
        ('was', 11410),
        ('it', 10681)]
    assert WORDS['the'] == 79809
    assert P('quintessential') == 0
    assert 0.07 < P('the') < 0.08
    return 'unit_tests pass'

def spelltest(tests, verbose=False):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    import time
    start = time.clock()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = correction(wrong)
        good += (w == right)
        if w != right:
            unknown += (right not in WORDS)
            if verbose:
                print('correction({}) => {} ({}); expected {} ({})'
                      .format(wrong, w, WORDS[w], right, WORDS[right]))
    dt = time.clock() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
          .format(good / n, n, unknown / n, n / dt))
    
def Testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

print(unit_tests())

unit_tests pass


In [28]:
spelltest(Testset(open('spell-testset1.txt'))) # Development set

75% of 270 correct (6% unknown) at 26 words per second 


In [29]:
spelltest(Testset(open('spell-testset2.txt'))) # Final test set

68% of 400 correct (11% unknown) at 22 words per second 


So on the development set we get 75% correct (processing words at a rate of 26 words/second), and on the final test set we get 68% correct (at 22 words/second). In conclusion, I met my goals for brevity, development time, and runtime speed, but not for accuracy. Perhaps my test set was extra tough, or perhaps my simple model is just not good enough to get to 80% or 90% accuracy.

### Future Work

Let's think about how we could do better. (I've developed the ideas some more in a [separate chapter](http://norvig.com/ngrams/) for a book and in a [Jupyter notebook](http://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb).)

1\. $P(c)$, the language model. We can distinguish two sources of error in the language model. The more serious is unknown words. In the development set, there are 15 unknown words, or 6%, and in the final test set, 43 unknown words or 11%. Here are some examples of the output of ```spelltest``` with ```verbose=True```:

In [39]:
spelltest(Testset(['transportability: transportibility']), verbose=True)

correction(transportibility) => transportibility (0); expected transportability (0)
0% of 1 correct (100% unknown) at 2 words per second 


In [41]:
spelltest(Testset(['addressable: addresable']), verbose=True)

correction(addresable) => addresable (0); expected addressable (0)
0% of 1 correct (100% unknown) at 4 words per second 


In [49]:
spelltest(Testset(['auxiliary: auxillary']), verbose=True)

correction(auxillary) => axillary (31); expected auxiliary (0)
0% of 1 correct (100% unknown) at 1551 words per second 


In this output we show the call to ```correction``` and the actual and expected result (with the ```WORDS``` counts in parentheses). Counts of (0) mean the target word was not in the dictionary, so we have no chance of getting it right. We could create a better language model by collecting more data, and perhaps by using a little English morphology (such as adding "ility" or "able" to the end of a word).

Another way to deal with unknown words is to allow the result of ```correction``` to be a word we have not seen. For example, if the input is "electroencephalographicallz", a good correction would be to change the final "z" to an "y", even though "electroencephalographically" is not in our dictionary. We could achieve this with a language model based on components of words: perhaps on syllables or suffixes, but it is easier to base it on sequences of characters: common 2-, 3- and 4-letter sequences.

2\. $P(w\,|\,c)$, the error model. So far, the error model has been trivial: the smaller the edit distance, the smaller the error. This causes some problems, as the examples below show. First, some cases where ```correction``` returns a word at edit distance 1 when it should return one at edit distance 2: 

In [61]:
spelltest(Testset(['receipt: reciet']), verbose=True)

correction(reciet) => recite (4); expected receipt (13)
0% of 1 correct (0% unknown) at 2571 words per second 


In [62]:
spelltest(Testset(['address: adres']), verbose=True)

correction(adres) => acres (36); expected address (76)
0% of 1 correct (0% unknown) at 2623 words per second 


In [63]:
spelltest(Testset(['remember: rember']), verbose=True)

correction(rember) => member (50); expected remember (161)
0% of 1 correct (0% unknown) at 2445 words per second 


Why should "adres" be corrected to "address" rather than "acres"? The intuition is that the two edits from "d" to "dd" and "s" to "ss" should both be fairly common, and have high probability, while the single edit from "d" to "c" should have low probability.

Clearly we could use a better model of the cost of edits. We could use our intuition to assign lower costs for doubling letters and changing a vowel to another vowel (as compared to an arbitrary letter change), but it seems better to gather data: to get a corpus of spelling errors, and count how likely it is to make each insertion, deletion, or alteration, given the surrounding characters. We need a lot of data to do this well. If we want to look at the change of one character for another, given a window of two characters on each side, that's 26<sup>6</sup>, which is over 300 million characters. You'd want several examples of each, on average, so we need at least a billion characters of correction data; probably safer with at least 10 billion.

Note there is a connection between the language model and the error model. The current program has such a simple error model (all edit distance 1 words before any edit distance 2 words) that it handicaps the language model: we are afraid to add obscure words to the model, because if one of those obscure words happens to be edit distance 1 from an input word, then it will be chosen, even if there is a very common word at edit distance 2. With a better error model we can be more aggressive about adding obscure words to the dictionary. Here are some examples where the presence of obscure words in the dictionary hurts us: 

In [55]:
spelltest(Testset(['wanted: wonted']), verbose=True)

correction(wonted) => wonted (1); expected wanted (213)
0% of 1 correct (0% unknown) at 4923 words per second 


In [48]:
spelltest(Testset(['planned: planed']), verbose=True)

correction(planed) => planed (1); expected planned (15)
0% of 1 correct (0% unknown) at 11082 words per second 


In [51]:
spelltest(Testset(['fourth: forth']), verbose=True)

correction(forth) => forth (82); expected fourth (76)
0% of 1 correct (0% unknown) at 7728 words per second 


3\. The enumeration of possible corrections, $\mathrm{argmax}_c$. Our program enumerates all corrections within edit distance 2. In the development set, only 3 words out of 270 are beyond edit distance 2, but in the final test set, there were 23 out of 400. Here they are:

    purple perpul
    curtains courtens
    minutes muinets

    successful sucssuful
    hierarchy heiarky
    profession preffeson
    weighted wagted
    inefficient ineffiect
    availability avaiblity
    thermawear thermawhere
    nature natior
    dissension desention
    unnecessarily unessasarily
    disappointing dissapoiting
    acquaintances aquantences
    thoughts thorts
    criticism citisum
    immediately imidatly
    necessary necasery
    necessary nessasary
    necessary nessisary
    unnecessary unessessay
    night nite
    minutes muiuets
    assessing accesing
    necessitates nessisitates

We could consider extending the model by allowing a limited set of edits at edit distance 3. For example, allowing only the insertion of a vowel next to another vowel, or the replacement of a vowel for another vowel, or replacing close consonants like "c" to "s" would handle almost all these cases.

4\. There's actually a fourth (and best) way to improve: change the interface to ```correction``` to look at more context. So far, ```correction``` only looks at one word at a time. It turns out that in many cases it is difficult to make a decision based only on a single word. This is most obvious when there is a word that appears in the dictionary, but the test set says it should be corrected to another word anyway: 

In [59]:
spelltest(Testset(['were: where']), verbose=True)

correction(where) => where (977); expected were (4289)
0% of 1 correct (0% unknown) at 9978 words per second 


In [53]:
spelltest(Testset(['later: latter']), verbose=True)

correction(latter) => latter (129); expected later (334)
0% of 1 correct (0% unknown) at 7844 words per second 


In [54]:
spelltest(Testset(['advise: advice']), verbose=True)

correction(advice) => advice (63); expected advise (19)
0% of 1 correct (0% unknown) at 5261 words per second 


Why should 'thear' be corrected as 'there' rather than 'their'? It is difficult to tell by the single word alone, but if the query were ```correction('There's no there thear')``` it would be clear.

To build a model that looks at multiple words at a time, we will need a lot of data. Fortunately, Google has released a [database of word counts](http://googleresearch.blogspot.com/2006/08/all-our-n-gram-are-belong-to-you.html) for all sequences up to five words long, gathered from a corpus of a trillion words.

I believe that a spelling corrector that scores 90% accuracy will _need_ to use the context of the surrounding words to make a choice. But we'll leave that for another day...

We could also decide what dialect we are trying to train for. The following three errors are due to confusion about American versus British spelling (our training data contains both): 

In [56]:
spelltest(Testset(['humour: humor']), verbose=True)

correction(humor) => humor (16); expected humour (4)
0% of 1 correct (0% unknown) at 7705 words per second 


In [57]:
spelltest(Testset(['organization: oranisation']), verbose=True)

correction(oranisation) => organisation (7); expected organization (42)
0% of 1 correct (0% unknown) at 1493 words per second 


In [58]:
spelltest(Testset(['organized: oranised']), verbose=True)

correction(oranised) => organised (10); expected organized (68)
0% of 1 correct (0% unknown) at 2038 words per second 


5\. Finally, we could improve the implementation by making it much faster, without changing the results. We could re-implement in a compiled language rather than an interpreted one. We could cache the results of computations so that we don't have to repeat them multiple times. One word of advice: before attempting any speed optimizations, profile carefully to see where the time is actually going. 